In [1]:
import pandas as pd
from matplotlib import font_manager
print("SimHei" in [f.name for f in font_manager.fontManager.ttflist])  

df = pd.read_csv('gender.csv').drop(columns=['Unnamed: 0'])
df

True


,gender_word,verb,relation,gender
0,妈妈,取名,nsubj,female
1,妈妈,顶,nsubj,female
2,爷爷,喜欢,nsubj,male
3,他们,像,nsubj,male
4,妈妈,与,pobj,female
...,...,...,...,...
4236,她,拧,nsubj,female
4237,她,替,pobj,female
4238,哥哥,知道,nsubj,male
4239,他,握,nsubj,male


In [ ]:
df[(df['gender']=='female') & (df['relation']=='dobj')]['verb'].value_counts()

verb
看     16
告诉    14
让     13
带     10
想      9
      ..
怕      1
如      1
宠爱     1
代      1
呵斥     1
Name: count, Length: 164, dtype: int64

In [31]:
df[(df['gender']=='male') & (df['relation']=='dobj')]['verb'].value_counts()

verb
看    21
让    15
问    13
叫    13
给    10
     ..
无     1
接     1
打     1
靠     1
住     1
Name: count, Length: 170, dtype: int64

In [32]:
df[(df['gender']=='male') & (df['relation']=='nsubj')]['verb'].value_counts()

verb
说     67
是     39
有     19
想     18
告诉    17
      ..
抽      1
思乡     1
喝酒     1
行经     1
跟着     1
Name: count, Length: 711, dtype: int64

In [33]:
df[(df['gender']=='female') & (df['relation']=='nsubj')]['verb'].value_counts()

verb
说     41
是     28
有     12
做     11
喜欢    11
      ..
建议     1
美      1
加上     1
熟      1
拧      1
Name: count, Length: 453, dtype: int64